In [ ]:
# Week3
# Problem1 Convert array into heap

def build_heap(data):
    """Build a heap from ``data`` inplace.

    Returns a sequence of swaps performed by the algorithm.
    """
    # The following naive implementation just sorts the given sequence
    # using selection sort algorithm and saves the resulting sequence
    # of swaps. This turns the given array into a heap, but in the worst
    # case gives a quadratic number of swaps.
    #
    # TODO: replace by a more efficient implementation
    swaps = []
    n = len(data)
    for i in range(n // 2, -1, -1):
        ShiftDown(i, data, swaps)
    
    return swaps

def ShiftDown(i, data, swaps):
    MaxIndex = i
    n = len(data)
    l = 2 * i + 1
    r = 2 * i + 2
    if l < n and data[l] < data[MaxIndex]:
        MaxIndex = l
    if r < n and data[r] < data[MaxIndex]:
        MaxIndex = r
    if i != MaxIndex:
        data[i], data[MaxIndex] = data[MaxIndex], data[i]
        swaps.append((i, MaxIndex))
        ShiftDown(MaxIndex, data, swaps)

def main():
    n = int(input())
    data = list(map(int, input().split()))
    assert len(data) == n

    swaps = build_heap(data)

    print(len(swaps))
    for i, j in swaps:
        print(i, j)


if __name__ == "__main__":
    main()

In [ ]:
# Problem2 Parallel processing

from collections import namedtuple

AssignedJob = namedtuple("AssignedJob", ["worker", "started_at"])


def assign_jobs(n_workers, jobs):
    results = []
    workers = []
    # initialize workers priority queue
    for i in range(n_workers):
        workers.append([0, i])
    
    BuildHeap(workers)
    
    for job in jobs:
        results.append(AssignedJob(workers[0][1], workers[0][0]))
        workers[0][0] += job # assign job to root 
        ShiftDown(0, workers)
    
    return results


def BuildHeap(A):
    n = len(A)
    for i in range(n // 2, -1, -1):
        ShiftDown(i, A)
    

def ShiftDown(i, A):
    MaxIndex = i
    n = len(A)
    l = 2 * i + 1
    r = 2 * i + 2
    if l < n:
        if A[l][0] < A[MaxIndex][0]:
            MaxIndex = l
        elif A[l][0] == A[MaxIndex][0] and A[l][1] < A[MaxIndex][1]:
            MaxIndex = l
        
    if r < n:
        if A[r][0] < A[MaxIndex][0]:
            MaxIndex = r
        elif A[r][0] == A[MaxIndex][0] and A[r][1] < A[MaxIndex][1]:
            MaxIndex = r
            
    if i != MaxIndex:
        A[i], A[MaxIndex] = A[MaxIndex], A[i]
        ShiftDown(MaxIndex, A)
    

def main():
    n_workers, n_jobs = map(int, input().split())
    jobs = list(map(int, input().split()))
    assert len(jobs) == n_jobs

    assigned_jobs = assign_jobs(n_workers, jobs)

    for job in assigned_jobs:
        print(job.worker, job.started_at)


if __name__ == "__main__":
    main()

In [12]:
# Problem3 Merging tables

class Database:
    def __init__(self, row_counts):
        self.row_counts = row_counts
        self.max_row_count = max(row_counts)
        n_tables = len(row_counts)
        self.ranks = [1] * n_tables
        self.parents = list(range(n_tables))

    def merge(self, dst, src):
        src_parent = self.get_parent(src)
        dst_parent = self.get_parent(dst)

        if src_parent == dst_parent:
            return False

        # merge two components
        # use union by rank heuristic
        # update max_row_count with the new maximum table size
        src_rank = self.ranks[src_parent]
        dst_rank = self.ranks[dst_parent]
        if src_rank < dst_rank:
            self.row_counts[dst_parent] += self.row_counts[src_parent]
            self.row_counts[src_parent] = 0
            self.parents[src_parent] = dst_parent
            if self.max_row_count < self.row_counts[dst_parent]:
                self.max_row_count = self.row_counts[dst_parent]
        else:
            self.row_counts[src_parent] += self.row_counts[dst_parent]
            self.row_counts[dst_parent] = 0
            self.parents[dst_parent] = src_parent
            if src_rank == dst_rank:
                self.ranks[src_parent] += 1
            if self.max_row_count < self.row_counts[src_parent]:
                self.max_row_count = self.row_counts[src_parent]
        
        return True

    def get_parent(self, table):
        # find parent and compress path
        #if table != self.parents[table]:
            #self.parents[table] = self.get_parent(self.parents[table])
        while table != self.parents[table]:
            table = self.parents[table]
        return table
    


def main():
    n_tables, n_queries = map(int, input().split())
    counts = list(map(int, input().split()))
    assert len(counts) == n_tables
    db = Database(counts)
    for i in range(n_queries):
        dst, src = map(int, input().split())
        db.merge(dst - 1, src - 1)
        print(db.max_row_count)
        #print(db.row_counts)
        #print(db.parents)


if __name__ == "__main__":
    main()

5 5
1 1 1 1 1
3 5
2
2 4
2
1 4
3
5 4
5
5 3
5


## 